In [49]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [13]:
%sendtofile --source "../Sensor_Kennel/BME280_funcs.py"

Sent 77 lines (3294 bytes) to BME280_funcs.py.


In [50]:
from machine import I2C, Pin
import time, os, sys, ustruct

#i2c = I2C(1)  # on Pi
i2c = I2C(1, scl=Pin(25), sda=Pin(27)) # on ESP32 pin26 broken


print(list(map(hex, i2c.scan())))  # 0x23=LTR-559, 0x76=BME280, 0x49=ADS1015

['0x23', '0x49', '0x76']


In [3]:
import BME280_funcs
BME280_funcs.bme280init(i2c, 0x76)
for i in range(20):
    x = BME280_funcs.readBME280(0x76)
    print("temp, baro, humid", x)
    time.sleep_ms(500)

temp, baro, humid (20.04, 99018.86, 40.09571)
temp, baro, humid (20.03, 99019.86, 40.10644)
temp, baro, humid (20.04, 99015.39, 40.09571)
temp, baro, humid (20.03, 99011.08, 40.08301)
temp, baro, humid (20.04, 99020.68, 40.07226)
temp, baro, humid (20.03, 99016.38, 40.07129)
temp, baro, humid (20.04, 99018.03, 40.11817)
temp, baro, humid (20.03, 99011.91, 40.1416)
temp, baro, humid (20.04, 99018.03, 40.15332)
temp, baro, humid (20.04, 99018.86, 40.1543)
temp, baro, humid (20.04, 99018.03, 40.12989)
temp, baro, humid (20.03, 99014.56, 40.12989)
temp, baro, humid (20.02, 99018.2, 40.10644)
temp, baro, humid (20.03, 99017.2, 40.10644)
temp, baro, humid (20.02, 99023.49, 40.09473)
temp, baro, humid (20.03, 99026.99, 40.07129)
temp, baro, humid (20.03, 99033.11, 40.07226)
temp, baro, humid (20.03, 99027.81, 40.07226)
temp, baro, humid (20.03, 99026.99, 40.08301)
temp, baro, humid (20.02, 99031.45, 40.05957)


In [59]:
print("LTR-559 part_number/revision", i2c.readfrom_mem(0x23, 0x86, 1), "should be 9 2")
i2c.writeto_mem(0x23, 0x80, b'\x09')  # enable ALS gain x4 (light sensor)
for i in range(20):
    x = ustruct.unpack("h", i2c.readfrom_mem(0x23, 0x8A, 2))
    print(x)
    time.sleep_ms(500)

LTR-559 part_number/revision b'\x92' should be 9 2
(3112,)
(3106,)
(3100,)
(3123,)
(3489,)
(4414,)
(3875,)
(3855,)
(4156,)
(3956,)
(3914,)
(3947,)
(3876,)
(44,)
(48,)
(47,)
(47,)
(47,)
(47,)
(47,)


In [22]:
# Numbers all seem wrong coming out here (should attach 3.3V to the heater enable pin)

# start conversion | channel | gain(FSR) | single shot | 1600 SPS | disable comparator
channels = [0x4000, 0x5000, 0x6000, 0x7000]  # in0/gnd, in1, in2, in3 (fixed at 1.25V reference)
gainfsr, gainV = 0x0000, 6.144
for i in range(20):
    vals = [ ]
    for channel in channels:
        i2c.writeto_mem(0x49, 0x01, ustruct.pack(">H", 0x8000 | channel | gainfsr | 0x0100 | 0x0080 | 0x0003))
        time.sleep_ms(10)
        #for j in range(10000):
        #    x = i2c.readfrom_mem(0x49, 0x01, 2)
        #    x1 = ustruct.unpack(">H", x)[0]
        #    if (x1 & 0x8000):  # conversion complete
        #        break
        r = ustruct.unpack(">h", i2c.readfrom_mem(0x49, 0x00, 2))
        #vals.append(r[0]>>3)
        v = (r[0] >> 3)/2048.0*gainV
        vals.append(v)
    print("ADS1015", vals)
    #print("ADS1015", "ox", 56000*(3.3-vals[0]), "red", 56000*(3.3-vals[1]), "nh3", 56000*(3.3-vals[2]), "ref", vals[3])
    time.sleep_ms(500)

ADS1015 [2.634, 5.814, 4.206, 1.062]
ADS1015 [2.634, 5.814, 4.206, 1.044]
ADS1015 [2.634, 5.808, 4.206, 1.2]
ADS1015 [2.634, 5.814, 4.206, 1.158]
ADS1015 [2.634, 5.814, 4.206, 1.044]
.ADS1015 [2.634, 5.814, 4.206, 1.056]
ADS1015 [2.634, 5.814, 4.206, 1.218]
ADS1015 [2.634, 5.814, 4.206, 1.074]
ADS1015 [2.634, 5.808, 4.206, 1.026]
ADS1015 [2.634, 5.808, 4.206, 1.146]
ADS1015 [2.634, 5.814, 4.206, 2.178]
ADS1015 [2.622, 5.802, 4.188, 1.164]
ADS1015 [2.616, 5.802, 4.176, 1.314]
ADS1015 [2.598, 5.796, 4.164, 2.778]
ADS1015 [2.592, 5.796, 4.158, 1.152]
.ADS1015 [2.592, 5.79, 4.158, 1.062]
ADS1015 [2.58, 5.79, 4.146, 1.422]
ADS1015 [2.586, 5.796, 4.158, 2.064]
ADS1015 [2.592, 5.796, 4.164, 1.296]
ADS1015 [2.598, 5.796, 4.17, 1.026]


In [97]:
# Dust sensor is UART, connect enable to 3V
from machine import UART
u = UART(1, tx=12, rx=14, baudrate=9600)
print(u)

UART(1, baudrate=9600, bits=8, parity=None, stop=1, tx=12, rx=14, rts=-1, cts=-1, txbuf=256, rxbuf=256, timeout=0, timeout_char=2)


In [139]:
import time, ustruct

cols = ["apm10", "apm25", "apm100", "pm10", "pm25", "pm100",
        "gt03um", "gt05um", "gt10um", "gt25um", "gt50um", "gt100um" ]
print("\t".join(cols))
for i in range(800):
    x = u.read()
    if x and len(x) == 32 and x[0] == 0x42 and x[1] == 0x4d:
        numbers = ustruct.unpack('>16H', x)
        csum = sum(x[:-2])
        if csum == numbers[-1]:
            assert(numbers[1] == 28) # frame length
            print("\t".join(map(str, numbers[2:-2])))
        else:
            print("bad checksum", x)
    time.sleep_ms(20)
        

apm10	apm25	apm100	pm10	pm25	pm100	gt03um	gt05um	gt10um	gt25um	gt50um	gt100um
1	1	1	1	1	1	1008	186	8	0	0	0
1	1	1	1	1	1	1008	186	8	0	0	0
1	1	2	1	1	2	1023	189	12	2	2	0
1	1	2	1	1	2	1023	189	12	2	2	0
1	1	2	1	1	2	1023	189	12	2	2	0
1	1	2	1	1	2	1050	191	14	2	2	0
2	2	3	2	2	3	1050	191	14	2	2	0
2	2	3	2	2	3	1050	191	14	2	2	0
2	2	4	2	2	4	1068	199	18	4	4	0
.2	2	4	2	2	4	1068	199	18	4	4	0
2	2	4	2	2	4	1068	199	18	4	4	0
2	3	5	2	3	5	1101	210	18	4	4	0
2	3	5	2	3	5	1101	210	18	4	4	0
2	3	5	2	3	5	1101	210	18	4	4	0
2	3	5	2	3	5	1089	208	18	4	4	0
2	3	5	2	3	5	1089	208	18	4	4	0
2	3	5	2	3	5	1089	208	18	4	4	0
2	3	5	2	3	5	1095	208	20	4	4	0
2	3	5	2	3	5	1095	208	20	4	4	0
.

In [68]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
Ready.


In [73]:
from machine import Pin, SPI
p = Pin(17, Pin.OUT)
p.value(1)

spi = SPI(1, baudrate=10000000, polarity=0, phase=0, sck=Pin(14, Pin), mosi=Pin(13), miso=Pin(12))
tft=TFT(spi, 16, None, 18)
#def __init__( self, spi, aDC, aReset, aCS) :

#GPIO10  SPIO MOSI    Pin13
#GPIO9   LCD D/C      Pin16
#GPIO11  SPIO SCLK    Pin14
#GPIO7   SPI CS       Pin18
#reset Pin17

tft.init_7735(tft.GREENTAB80x160)
for i in range(10):
    tft.rect((10+i*5,10),(40,40+i*2), TFT.WHITE)
         
        
#disp = ST7735.ST7735(
#        port=0, cs=1, dc=9, backlight=12, rotation=270, spi_speed_hz=10000000
#    )

In [11]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [12]:
ST7735_TFTWIDTH = 80
ST7735_TFTHEIGHT = 160

import framebuf, time
fdata = bytearray(ST7735_TFTWIDTH*ST7735_TFTHEIGHT*2)
fbuf = framebuf.FrameBuffer(fdata, ST7735_TFTWIDTH, ST7735_TFTHEIGHT, framebuf.RGB565)


In [6]:
#disp = ST7735(1, Pin(18, Pin.OUT), Pin(16, Pin.OUT), backlight=Pin(17, Pin.OUT), rst=None, rotation=270)


In [13]:

from machine import Pin, SPI

width = 80
height = 160

import framebuf, time
fdata = bytearray(width*height*2)
fbuf = framebuf.FrameBuffer(fdata, width, height, framebuf.RGB565)

spi = SPI(1, baudrate=10000000, polarity=0, phase=0, sck=Pin(14), mosi=Pin(13), miso=Pin(12))
cs = Pin(18, Pin.OUT)
dc = Pin(16, Pin.OUT)
backlight = Pin(17, Pin.OUT)

ST7735_COLS = 132
ST7735_ROWS = 162

def color565(r, g, b):
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)

backlight.value(0)
time.sleep(0.1)
backlight.value(1)

def cmddata(cmd, data=None):
    cs.value(0)
    dc.value(0)
    spi.write(bytearray([cmd]))
    if data:
        dc.value(1)
        spi.write(data)
    cs.value(1)


In [15]:
import time

offset_left = (ST7735_COLS - width) // 2
offset_top = (ST7735_ROWS - height) // 2
print(offset_left, offset_top)


# Initialize the display.
cmddata(0x01)# Software reset
time.sleep(0.150)               # delay 150 ms
cmddata(0x11)# Out of sleep mode
time.sleep(0.500)               # delay 500 ms

cmddata(0xB1, bytearray([0x01, 0x2C, 0x2D]))    # Frame rate ctrl - normal mode
                 # Rate = fosc/(1x2+40) * (LINE+2C+2D)
cmddata(0xB2, bytearray([0x01, 0x2C, 0x2D]))    # Frame rate ctrl - idle mode
                 # Rate = fosc/(1x2+40) * (LINE+2C+2D)

cmddata(0xB3, bytearray([0x01, 0x2C, 0x2D, 0x01, 0x2C, 0x2D]))    # Frame rate ctrl - partial mode
                # Dot inversion mode
                 # Line inversion mode

cmddata(0xB4, bytearray([0x07]))     # Display inversion ctrl
                 # No inversion

cmddata(0xC0, bytearray([0xA2, 0x02, 0x84]))     # Power control
                 # -4.6V
                 # auto mode

cmddata(0xC1, bytearray([0x0A, 0x00]))     # Power control
         # Opamp current small
         # Boost frequency

cmddata(0xC3, bytearray([0x8A, 0x2A]))     # Power control
                 # BCLK/2, Opamp current small & Medium low


cmddata(0xC4, bytearray([0x8A, 0xEE]))     # Power control
cmddata(0xC5, bytearray([0x0E]))     # Power control
cmddata(0x21)  # Don't invert display

cmddata(0x36, bytearray([0xC8]))     # Memory access control (directions)
                 # row addr/col addr, bottom to top refresh

cmddata(0x3A, bytearray([0x05]))     # set color mode
                 # 16-bit color

cmddata(0x2A, bytearray([0x00, offset_left, 0x00, width + offset_left - 1]))      # Column addr set
                 # XSTART = 0
                 # XEND = ROWS - height

cmddata(0x2B, bytearray([0x00, offset_top, 0x00, height + offset_top - 1]))      # Row addr set
                 # XSTART = 0
                 # XEND = COLS - width

cmddata(0xE0, bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29, 0x25, 0x2B, 0x39, 0x00, 0x01, 0x03, 0x10]))    # Set Gamma
cmddata(0xE1, bytearray([0x03, 0x1d, 0x07, 0x06, 0x2E, 0x2C, 0x29, 0x2D, 0x2E, 0x2E, 0x37, 0x3F, 0x00, 0x00, 0x02, 0x10]))    # Set Gamma

cmddata(0x13)      # Normal display on
time.sleep(0.10)                # 10 ms
cmddata(0x29)     # Display on
time.sleep(0.100)               # 100 ms


26 1


In [16]:
def display(fdata):
    x1 = width - 1
    y1 = height - 1
    y0 = offset_top
    y1 += offset_top
    x0 = offset_left
    x1 += offset_left
    cmddata(0x2A, bytearray([x0 >> 8, x0, x1 >> 8, x1]))       # Column addr set
                # XSTART
                        # XEND
    cmddata(0x2B, bytearray([y0 >> 8, y0, y1>>8, y1]))       # Row addr set
                        # YSTART
                        # YEND
    cmddata(0x2C, fdata)       # write to RAM


In [17]:
fbuf.fill(0x00f0)
for i in range(10):
    fbuf.fill_rect(10+i*5,i*10,8,8,0x0000)
    display(fdata)
    time.sleep(0.5)
fbuf.text("hi there", 10,100,0xffff)
display(fdata)

.

In [5]:

from machine import Pin, SPI

width = 80
height = 160

import framebuf, time
fdata = bytearray(width*height*2)
fbuf = framebuf.FrameBuffer(fdata, width, height, framebuf.RGB565)


ST7735_COLS = 132
ST7735_ROWS = 162

def color565(r, g, b):
    """Convert red, green, blue components to a 16-bit 565 RGB value. Components
    should be values 0 to 255.
    """
    return ((r & 0xF8) << 8) | ((g & 0xFC) << 3) | (b >> 3)

class ST7735(object):
    """Representation of an ST7735 TFT LCD."""

    def __init__(self, port, cs, dc, backlight=None, rst=None, width=ST7735_TFTWIDTH,
                 height=ST7735_TFTHEIGHT, rotation=90, offset_left=None, offset_top=None, invert=True, spi_speed_hz=4000000):

        self._spi = SPI(1, baudrate=10000000, polarity=0, phase=0, sck=Pin(14), mosi=Pin(13), miso=Pin(12))
        self._dc = dc
        self._rst = rst
        self._width = width
        self._height = height
        self._rotation = rotation
        self._invert = invert
        self._cs = Pin(18, Pin.OUT)
        
        # Default left offset to center display
        if offset_left is None:
            offset_left = (ST7735_COLS - width) // 2

        self._offset_left = offset_left

        # Default top offset to center display
        if offset_top is None:
            offset_top = (ST7735_ROWS - height) // 2

        self._offset_top = offset_top

        # Setup backlight as output (if provided).
        self._backlight = backlight
        if backlight is not None:
            backlight.value(0)
            time.sleep(0.1)
            backlight.value(1)
            
        self._init()

    def cmddata(self, cmd, data=None):
        self._cs.value(0)
        self._dc.value(0)
        self._spi.write(bytearray([cmd]))
        if data:
            self._dc.value(1)
            self._spi.write(data)
        self._cs.value(1)
        
    def _init(self):
        # Initialize the display.
        self.cmddata(0x01, None)# Software reset
        time.sleep(0.150)               # delay 150 ms
        self.cmddata(0x11, None)# Out of sleep mode
        time.sleep(0.500)               # delay 500 ms

        self.cmddata(0xB1, bytearray([0x01, 0x2C, 0x2D]))    # Frame rate ctrl - normal mode
                         # Rate = fosc/(1x2+40) * (LINE+2C+2D)
        self.cmddata(0xB2, bytearray([0x01, 0x2C, 0x2D]))    # Frame rate ctrl - idle mode
                         # Rate = fosc/(1x2+40) * (LINE+2C+2D)

        self.cmddata(0xB3, bytearray([0x01, 0x2C, 0x2D, 0x01, 0x2C, 0x2D]))    # Frame rate ctrl - partial mode
                        # Dot inversion mode
                         # Line inversion mode

        self.cmddata(0xB4, bytearray([0x07]))     # Display inversion ctrl
                         # No inversion
            
        self.cmddata(0xC0, bytearray([0xA2, 0x02, 0x84]))     # Power control
                         # -4.6V
                         # auto mode

        self.cmddata(0xC1, bytearray([0x0A, 0x00]))     # Power control
                 # Opamp current small
                 # Boost frequency

        self.cmddata(0xC3, bytearray([0x8A, 0x2A]))     # Power control
                         # BCLK/2, Opamp current small & Medium low
        

        self.cmddata(0xC4, bytearray([0x8A, 0xEE]))     # Power control
        self.cmddata(0xC5, bytearray([0x0E]))     # Power control
        self.cmddata(0x21)  # Don't invert display

        self.cmddata(0x36, bytearray([0xC8]))     # Memory access control (directions)
                         # row addr/col addr, bottom to top refresh

        self.cmddata(0x3A, bytearray([0x05]))     # set color mode
                         # 16-bit color

        self.cmddata(0x2A, bytearray([0x00, self._offset_left, 0x00, self._width + self._offset_left - 1]))      # Column addr set
                         # XSTART = 0
                         # XEND = ROWS - height
        
        self.cmddata(0x2B, bytearray([0x00, self._offset_top, 0x00, self._height + self._offset_top - 1]))      # Row addr set
                         # XSTART = 0
                         # XEND = COLS - width

        self.cmddata(0xE0, bytearray([0x02, 0x1c, 0x07, 0x12, 0x37, 0x32, 0x29, 0x2d, 0x29, 0x25, 0x2B, 0x39, 0x00, 0x01, 0x03, 0x10]))    # Set Gamma
        self.cmddata(0xE1, bytearray([0x03, 0x1d, 0x07, 0x06, 0x2E, 0x2C, 0x29, 0x2D, 0x2E, 0x2E, 0x37, 0x3F, 0x00, 0x00, 0x02, 0x10]))    # Set Gamma

        self.cmddata(0x13)      # Normal display on
        time.sleep(0.10)                # 10 ms

        self.cmddata(0x29)     # Display on
        time.sleep(0.100)               # 100 ms


    def display(self, fdata):
        ST7735_TFTWIDTH = 80
        ST7735_TFTHEIGHT = 160
        ST7735_COLS = 132
        ST7735_ROWS = 162
        width=ST7735_TFTWIDTH
        height=ST7735_TFTHEIGHT
        offset_left = (ST7735_COLS - width) // 2
        offset_top = (ST7735_ROWS - height) // 2
        x1 = width - 1
        y1 = height - 1
        y0 = offset_top
        y1 += offset_top
        x0 = offset_left
        x1 += offset_left

        self.cmddata(0x2A, bytearray([x0 >> 8, x0, x1 >> 8, x1]))       # Column addr set
                    # XSTART
                            # XEND
        self.cmddata(0x2B, bytearray([y0 >> 8, y0, y1>>8, y1]))       # Row addr set
                            # YSTART
                            # YEND
        self.cmddata(0x2C, fdata)       # write to RAM



In [109]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


object <module 'framebuf'> is of type module
  __name__ -- framebuf
  FrameBuffer -- <class 'FrameBuffer'>
  FrameBuffer1 -- <function>
  MVLSB -- 0
  MONO_VLSB -- 0
  RGB565 -- 1
  GS2_HMSB -- 5
  GS4_HMSB -- 2
  GS8 -- 6
  MONO_HLSB -- 3
  MONO_HMSB -- 4
